## MLP

We have checked out cajamar website and there is clearly three categories of products they sell. 

- Particulares: Products aimed at 'particulares' 

- Empresa: Products aimed at bussiness

- ADN Agro: Products aimed at farmers

Socio_Demo_05: Segmento (00: Particular | 01:Agricultor | 02:Comercio | 03:Autónomo)


A quick google search of 'Cajamar autonomos' tells us they offer products exclusively tailored for 'autonomos'.

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# set fixed size for plots
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [4]:
data = pd.read_csv('train2-2.txt', sep='|')
dftest = pd.read_csv('test2.txt', sep= '|')
data.head()

,ID_Customer,Cod_Prod,Cod_Fecha,Socio_Demo_01,Socio_Demo_02,Socio_Demo_03,Socio_Demo_04,Socio_Demo_05
0,A0000001,601,2007-05,5,4,3,1,0
1,A0000001,704,2013-04,5,4,3,1,0
2,A0000001,2501,2006-03,5,4,3,1,0
3,A0000001,2503,2006-03,5,4,3,1,0
4,A0000001,1011,2011-04,5,4,3,1,0


In [6]:
data.groupby('Socio_Demo_05')['ID_Customer'].count()

Socio_Demo_05
0    2657685
1     163495
2     155285
3     374136
Name: ID_Customer, dtype: int64

As expected the number of sells to 'particulares' is one order of magnitude bigger

Lets explore who bought what.

In [11]:
prod_sells_by_S5= data.groupby(['Socio_Demo_05', 'Cod_Prod'])['ID_Customer'].count()
prod_sells_by_S5.head()

Socio_Demo_05  Cod_Prod
0              101            123
               102            829
               103           1323
               104             52
               201         271312
Name: ID_Customer, dtype: int64

In [18]:
prod_sells_by_S5.unstack()

Cod_Prod,101,102,103,104,201,301,502,503,504,506,...,2706,2707,2801,2901,3001,3101,3401,9991,9992,9993
Socio_Demo_05,,,,,,,,,,,,,,,,,,,,,
0,123.0,829.0,1323.0,52.0,271312.0,349287.0,2.0,931.0,27.0,300.0,...,5752.0,23523.0,15.0,1.0,8444.0,440.0,3059.0,196601.0,15688.0,196601.0
1,17.0,127.0,314.0,9.0,11201.0,18218.0,NaN,1637.0,4.0,31.0,...,217.0,1982.0,2.0,NaN,400.0,35.0,1015.0,8240.0,15831.0,8240.0
2,30.0,581.0,279.0,4.0,16724.0,17239.0,NaN,88.0,4.0,58.0,...,297.0,2057.0,6.0,2.0,688.0,33.0,908.0,5418.0,10999.0,5418.0
3,91.0,669.0,911.0,40.0,40449.0,41425.0,1.0,203.0,34.0,280.0,...,723.0,3725.0,11.0,3.0,1652.0,78.0,2781.0,20164.0,29144.0,20164.0


There are some products that barerly sell, taking into account that the least ammount of sells we have in a particular category of Socio_Dem5 is 15K, we think we can safely ignore products with less than 300 sells, they do not represent a siginificant percentage of the market and will barerly affect accuracy. Given the vast ammount of them (23/94) that will significally help our model as we avoid 'curse of dimensionality' (Mirar wiki explicar bé)

In [22]:
Sells_per_product = data.groupby('Cod_Prod').ID_Customer.count()

products_sub300=Sells_per_product[Sells_per_product < 300].index
products_sub300.size

23

In [24]:
#Filter data deleting the rare products
filt_data = data.query('Cod_Prod not in @products_sub300')
prod_sells_by_S5= filt_data.groupby(['Socio_Demo_05', 'Cod_Prod'])['ID_Customer'].count()

In [151]:
#Filter data deleting the Customers that bought only one product. We cant use them for train
#Clean the datafram of useless users.
dropusers = filt_data.groupby('ID_Customer')['Cod_Prod'].count()
dropusers = dropusers[dropusers == 1]
filt_data = filt_data.query('ID_Customer not in @dropusers.index')

In [153]:
filt_data.head()

,ID_Customer,Cod_Prod,Cod_Fecha,Socio_Demo_01,Socio_Demo_02,Socio_Demo_03,Socio_Demo_05
0,A0000001,601,2007-05,5,4,3,0
1,A0000001,704,2013-04,5,4,3,0
2,A0000001,2501,2006-03,5,4,3,0
3,A0000001,2503,2006-03,5,4,3,0
4,A0000001,1011,2011-04,5,4,3,0


In [25]:
prod_sells_by_S5.head()

Socio_Demo_05  Cod_Prod
0              102            829
               103           1323
               201         271312
               301         349287
               503            931
Name: ID_Customer, dtype: int64

In [28]:
prod_sells_by_S5 =prod_sells_by_S5.unstack()

In [29]:
prod_sells_by_S5

Cod_Prod,102,103,201,301,503,506,601,702,704,705,...,2704,2705,2706,2707,3001,3101,3401,9991,9992,9993
Socio_Demo_05,,,,,,,,,,,,,,,,,,,,,
0,829,1323,271312,349287,931,300,564472,4270,80743,739,...,80572,43809,5752,23523,8444,440,3059,196601,15688,196601
1,127,314,11201,18218,1637,31,22017,142,2846,30,...,2950,1317,217,1982,400,35,1015,8240,15831,8240
2,581,279,16724,17239,88,58,21638,173,2195,20,...,5056,2022,297,2057,688,33,908,5418,10999,5418
3,669,911,40449,41425,203,280,53629,446,6025,124,...,13149,4886,723,3725,1652,78,2781,20164,29144,20164


In [32]:
prod_sells_by_S5.columns

Int64Index([ 102,  103,  201,  301,  503,  506,  601,  702,  704,  705,  706,
             707,  708,  801, 1001, 1002, 1007, 1009, 1011, 1012, 1013, 1017,
            1019, 1020, 1021, 1022, 1301, 1302, 1303, 1304, 1306, 1307, 1309,
            1310, 1311, 1401, 1501, 1801, 1802, 1804, 1805, 2102, 2103, 2105,
            2106, 2201, 2202, 2203, 2204, 2205, 2206, 2301, 2302, 2401, 2501,
            2503, 2601, 2602, 2701, 2702, 2703, 2704, 2705, 2706, 2707, 3001,
            3101, 3401, 9991, 9992, 9993],
           dtype='int64', name=u'Cod_Prod')

In [37]:
#We now normalize prod_sells_by_S5, 
#so we can see where do this products perform better and if as expected it is an important aspect
prod_sells_by_S5norm = prod_sells_by_S5.copy()
for i in prod_sells_by_S5norm.columns:
    prod_sells_by_S5norm[i] = prod_sells_by_S5norm[i]/prod_sells_by_S5norm[i].sum()
    

In [95]:
prod_sells_by_S5norm[prods[0:20]]

Cod_Prod,102,103,201,301,503,506,601,702,704,705,706,707,708,801,1001,1002,1007,1009,1011,1012
Socio_Demo_05,,,,,,,,,,,,,,,,,,,,
0,0.375793,0.467987,0.798714,0.819597,0.325638,0.448430,0.852991,0.848738,0.879467,0.809419,0.895203,0.902055,0.895450,0.525889,0.529527,0.788387,0.684449,0.184043,0.808925,0.313620
1,0.057570,0.111072,0.032975,0.042748,0.572578,0.046338,0.033271,0.028225,0.030999,0.032859,0.026721,0.022664,0.019301,0.120878,0.155065,0.073118,0.037745,0.021986,0.041535,0.469534
2,0.263373,0.098691,0.049234,0.040451,0.030780,0.086697,0.032698,0.034387,0.023908,0.021906,0.019365,0.020770,0.025276,0.079663,0.107255,0.040323,0.089079,0.717376,0.044471,0.082437
3,0.303264,0.322250,0.119078,0.097203,0.071004,0.418535,0.081040,0.088650,0.065625,0.135816,0.058711,0.054511,0.059972,0.273570,0.208154,0.098172,0.188727,0.076596,0.105069,0.134409


Looking at the data we see that there is huge variability in popularity of the products depending on on which category of S5 we look at.

   **HEM DE FER UN PLOT DE PROD VS SELLS A PARTIR DE prod_sells_byS5norm** Recordar que hi han molts mes clients S5= 0, per tant estara un pel esbiaxiat peor igualment ens serivrà. 

In [87]:
rcParams['figure.figsize'] = 5, 50
rows=4
cols=3
f, ax2= plt.subplots(rows,cols, sharex=False, sharey=True )
colors = ['#0000FF','#4682B4','#00FFFF','#008000','#9ACD32','#EEE8AA','#FFD700','#FFA500','#FF4500','#B22222'] 
prods=prod_sells_by_S5norm.columns
for i in range(prods.size/10+1):
        for j in range(6):
            if 10*i+j < products.size:
                ax2[i,0]=plt.bar(prod_sells_by_S5norm.index.values,prod_sells_by_S5norm[prods[i]] ,color=colors[j], label=str(prods))

IndexError: index 4 is out of bounds for axis 0 with size 4

In [82]:
prods.size/4

17

In [83]:
rcParams['figure.figsize'] = 15, 40
rows=10
cols=5
f, ax2= plt.subplots(rows, cols, sharex=False, sharey=False)
colors = ['#0000FF','#4682B4','#00FFFF','#008000','#9ACD32','#EEE8AA','#FFD700','#FFA500','#FF4500','#B22222'] 
products=prod_sells_by_S5norm.columns
for i in range(0, products.size/10+1):
    for j in range(10):
        if 10*i+j < products.size: #Products definido en la celda de arriba
            product = products[10*i+j]
            ax2[i,0].plot(prod_sells_by_S5norm.index.values, prod_sells_by_S5norm[products[j]], color=colors[j], label=str(product))
    #leyenda for k in range(5):

In [46]:
prod_sells_by_S5.index.values

array([0, 1, 2, 3])

In [50]:
for i in range(0, products.size/10+1):
    for j in range(10):
        if 10*i+j < products.size: #Products definido en la celda de arriba
            product = products[10*i+j]
            df=product_rel(data, product)
            ax2[i,0].plot(df.index.values, df.Socio_Demo_01, color=colors[j], label=str(product))
            ax2[i,1].plot(df.index.values, df.Socio_Demo_02, color=colors[j], label=str(product))
            ax2[i,2].plot(df.index.values, df.Socio_Demo_03, color=colors[j], label=str(product))
            ax2[i,3].plot([1,2], df.Socio_Demo_04[:2], color=colors[j], label=str(product))
            ax2[i,4].plot(range(4), df.Socio_Demo_05[:4], color=colors[j], label=str(product))
    #leyenda for k in range(5):

NameError: name 'products' is not defined

In [154]:
filt_data.drop('Socio_Demo_04', axis=1,inplace= True)
filt_data.head()

ValueError: labels ['Socio_Demo_04'] not contained in axis

Excluyendo el sexo por razones eticas. Para más informacion sobre nuestra decision consultar https://obamawhitehouse.archives.gov/blog/2016/05/04/big-risks-big-opportunities-intersection-big-data-and-civil-rights



Dividimos los datos en 4 tipos segun su S5

In [161]:
grouped =filt_data.groupby('ID_Customer')['Cod_Prod'].count()
grouped[grouped ==1 ]

Series([], Name: Cod_Prod, dtype: int64)

In [162]:
#We split the data set into subcategories acording to its S5

data_0 = filt_data.query('Socio_Demo_05 == 0')
data_1 = filt_data.query('Socio_Demo_05 == 1')
data_2 = filt_data.query('Socio_Demo_05 == 2')
data_3 = filt_data.query('Socio_Demo_05 == 3')


In [163]:
print data_0.shape
print data_1.shape
print data_2.shape
filt_data.groupby('Socio_Demo_05')['ID_Customer'].count()

#Todo bien

(2585119, 7)
(163078, 7)
(154620, 7)


Socio_Demo_05
0    2585119
1     163078
2     154620
3     372481
Name: ID_Customer, dtype: int64

In [108]:
data_0.head()

,ID_Customer,Cod_Prod,Cod_Fecha,Socio_Demo_01,Socio_Demo_02,Socio_Demo_03,Socio_Demo_05
0,A0000001,601,2007-05,5,4,3,0
1,A0000001,704,2013-04,5,4,3,0
2,A0000001,2501,2006-03,5,4,3,0
3,A0000001,2503,2006-03,5,4,3,0
4,A0000001,1011,2011-04,5,4,3,0


Entrenaremos una red neuronal para cada grupo de datos, la entrada sera, el vector de productos comprados con 0 si no lo ha contratado y 1 si sí, ademas añadiremos las features Socio_Demo en forma one-hot.

Para entrenar la red hay que dividir los datos en features i target, necesitamos eliminar la ultima compra de cada usuario i trasladar-lo al target

In [199]:
def customer_buys(data):
    """
        Crea la matriz idVSProd, nos da por cada usuario el vector de compra de productos 
        retorna dataframe (numero_customers, numero_productos)
    """
    colnames = data.Cod_Prod.unique()
    colnames.sort()
    rownames = data.ID_Customer.unique()
    rownames.sort()
    idVSProd = pd.DataFrame(data=np.zeros((data.ID_Customer.unique().size,data.Cod_Prod.unique().size)),columns=colnames,index=rownames,dtype=int)
    groupedData = data.groupby(['ID_Customer','Cod_Prod'])['Cod_Prod'].count()
    for product in colnames:
        columna = groupedData[:,product]
        idVSProd.loc[columna.index,product] = columna.values
    return idVSProd

In [128]:
def last_prod(df):
    '''
    returns a Series with indexes ID_customer values last product that customer bought
    '''
    data = df.copy()
    data.sort_values('Cod_Fecha', inplace=True)
    data_lastp = data.groupby('ID_Customer').last()
    
    return data_lastp.Cod_Prod

In [197]:
data_0.ID_Customer.unique().shape

(504637,)

In [149]:
%%time
#Borramos el ultimo producto Aqui estamos borrando ultimo producto de primer usuario, esto es el 704. Todo bien
cust_prods_0.loc[data_0_lastp.index[0], data_0_lastp.ix[data_0_lastp.index[0]]] = 0
cust_prods_0.ix[data_0_lastp.index[0]].head(10)

CPU times: user 974 µs, sys: 144 µs, total: 1.12 ms
Wall time: 990 µs


In [170]:
#Delete the last buys from the cust_prods and create the input
#ESTO TARDA 12 MIN, no hay mejor manera de hacerlo ???
def create_X(cust_prods, data):
    """
    Devuelve el dataframe cust_buys (idVSProd) sin el ultimo producto
    
    cust_prods: és la matriz de compra de cada producto idVSprod
    data: es el dataframe orignal /filtrado etc
    """
#Compute the last products vector
    train = cust_prods.copy()
    last_product = last_prod(data) #It returns a Series with users as index and last product they bought as values
    for i in last_product.index:
        train.loc[i, last_product.ix[i]] = 0
    return train

In [198]:
#Ceamos la matriz de compra de productos, por cada usuario.
cust_prods_0 = customer_buys(data_0)

In [200]:
cust_prods_0.head()

,102,103,201,301,503,506,601,702,704,705,...,2704,2705,2706,2707,3001,3101,3401,9991,9992,9993
A0000001,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
A0000002,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A0000003,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0000004,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0000005,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
X_0 = create_X(cust_prods_0, data_0)

In [208]:
X_0.ix['A0000002'].sum()

3

We have yet to add the features of every user. We shall use a one-hot encoding

In [173]:
data_0.head()

,ID_Customer,Cod_Prod,Cod_Fecha,Socio_Demo_01,Socio_Demo_02,Socio_Demo_03,Socio_Demo_05
0,A0000001,601,2007-05,5,4,3,0
1,A0000001,704,2013-04,5,4,3,0
2,A0000001,2501,2006-03,5,4,3,0
3,A0000001,2503,2006-03,5,4,3,0
4,A0000001,1011,2011-04,5,4,3,0


In [176]:
data_0[['Socio_Demo_01', 'Socio_Demo_02']].ix[0:2]

,Socio_Demo_01,Socio_Demo_02
0,5,4
1,5,4
2,5,4


In [184]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [227]:
feature_matrix.shape
del y_cat, y

In [255]:
y1-1

array([4, 4, 4, ..., 0, 2, 2])

In [268]:
# Transform all values to categorical values. One-hot encoding.

feature_matrix = data_0.groupby('ID_Customer')[['Socio_Demo_01', 'Socio_Demo_02','Socio_Demo_03','Socio_Demo_05']].first()

#Prepare Socio_Demo_01
y1 = feature_matrix['Socio_Demo_01'].values
#Restamos uno para tener el numero apropiado de classes, de 0 a 4. 5.
y1 = y1 -1 
y1_cat = to_categorical(y1, nb_classes=5)
y1df= pd.DataFrame(y1_cat, index = feature_matrix.index.values, columns= ['1.1','1.2','1.3', '1.4', '1.5'])

#Prepare Socio_Demo_02
y2 = feature_matrix['Socio_Demo_02'].values
y2 = y2 -1 
y2_cat = to_categorical(y2, nb_classes=5)
y2df= pd.DataFrame(y2_cat, index = feature_matrix.index.values, columns= ['2.1','2.2','2.3', '2.4', '2.5'])

#Prepare Socio_Demo_03
y3 = feature_matrix['Socio_Demo_03'].values
y3 = y3 - 1
y3_cat = to_categorical(y3, nb_classes = 5)
y3df= pd.DataFrame(y3_cat, index = feature_matrix.index.values, columns= ['3.1','3.2','3.3', '3.4', '3.5'])


In [269]:
#Concatenate all the dataframes

X_0_train = pd.concat([X_0,y1df,y2df,y3df], axis=1)

In [246]:
X_0.shape

(504637, 71)

In [270]:
X_0_train.shape

(504637, 86)

In [248]:
filt_data.Socio_Demo_03.unique().size

5

In [257]:
print 'Deberiamos tener: ', cust_prods_0.columns.size, '+', filt_data.Socio_Demo_01.unique().size, '+', filt_data.Socio_Demo_02.unique().size, '+',filt_data.Socio_Demo_03.unique().size, '='
print 'Tenemos: ',cust_prods_0.columns.size, '+', y1_cat.shape[1] , '+' ,y2_cat.shape[1], '+' ,y3_cat.shape[1] 
print cust_prods_0.columns.size + filt_data.Socio_Demo_01.unique().size + filt_data.Socio_Demo_02.unique().size + filt_data.Socio_Demo_03.unique().size 

Deberiamos tener:  71 + 5 + 5 + 5 =
Tenemos:  71 + 5 + 5 + 5
86


Parece que todo cuadra. ok

In [280]:
X_0_train.loc[:,'1.1':'3.5'].head(10)

,1.1,1.2,1.3,1.4,1.5,2.1,2.2,2.3,2.4,2.5,3.1,3.2,3.3,3.4,3.5
A0000001,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
A0000002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
A0000003,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
A0000004,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
A0000005,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
A0000006,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
A0000007,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
A0000009,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
A0000010,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
A0000011,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [281]:
feature_matrix.head(10)

,Socio_Demo_01,Socio_Demo_02,Socio_Demo_03,Socio_Demo_05
ID_Customer,,,,
A0000001,5,4,3,0
A0000002,5,5,1,0
A0000003,5,5,5,0
A0000004,5,5,3,0
A0000005,5,5,3,0
A0000006,5,5,3,0
A0000007,5,5,2,0
A0000009,4,5,2,0
A0000010,5,5,2,0


Todo parece que funciona ok! podemos empezar a diseñar los conjuntos de entreno.

,102,103,201,301,503,506,601,702,704,705,...,2.1,2.2,2.3,2.4,2.5,3.1,3.2,3.3,3.4,3.5
A0212343,0,0,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
A0229787,0,0,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
A0214756,0,0,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
A0115626,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
A0313809,0,0,0,1,0,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [285]:
#Split into train and test
from sklearn.cross_validation import train_test_split

train0, test0 = train_test_split(X_0_train, test_size = 0.2)

In [332]:
train0.index.name = 'ID_Customer'
test0.index.name = 'ID_Customer'
train0.sort_index( inplace= True)
test0.sort_index(inplace= True)
train0.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,102,103,201,301,503,506,601,702,704,705,...,2.1,2.2,2.3,2.4,2.5,3.1,3.2,3.3,3.4,3.5
ID_Customer,,,,,,,,,,,,,,,,,,,,,
A0000002,0,0,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
A0000003,0,0,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
A0000005,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
A0000009,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
A0000011,0,0,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [333]:
#Create nested arrays from dataframe
X0_train = np.array(train0.values.tolist(), dtype = int)
X0_test = np.array (test0.values.tolist(), dtype = int)

In [318]:
cust_prods_0.index.name = 'ID_Customer'
cust_prods_0.head()

,102,103,201,301,503,506,601,702,704,705,...,2704,2705,2706,2707,3001,3101,3401,9991,9992,9993
ID_Customer,,,,,,,,,,,,,,,,,,,,,
A0000001,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
A0000002,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A0000003,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0000004,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0000005,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [334]:
target0_train = cust_prods_0.query('ID_Customer in @train0.index.values')
target0_test = cust_prods_0.query('ID_Customer in @test0.index.values')
y0_train = np.array(target0_train.values.tolist(), dtype = float)
y0_test = np.array(target0_test.values.tolist(), dtype =float)

In [343]:
print y0_train[0].sum()
print X0_train[0][:72].sum()

4.0
3.0


In [340]:
last_products0 = last_prod(data_0)

In [341]:
last_products0[:3]

ID_Customer
A0000001     704
A0000002    9992
A0000003     201
Name: Cod_Prod, dtype: int64

In [339]:
target0_train.ix[0]

102     0
103     0
201     0
301     1
503     0
506     0
601     1
702     0
704     0
705     0
706     0
707     0
708     0
801     1
1001    0
1002    0
1007    0
1009    0
1011    0
1012    0
1013    0
1017    0
1019    0
1020    0
1021    0
1022    0
1301    0
1302    0
1303    0
1304    0
       ..
2102    0
2103    0
2105    0
2106    0
2201    0
2202    0
2203    0
2204    0
2205    0
2206    0
2301    0
2302    0
2401    0
2501    0
2503    0
2601    0
2602    0
2701    0
2702    0
2703    0
2704    0
2705    0
2706    0
2707    0
3001    0
3101    0
3401    0
9991    0
9992    1
9993    0
Name: A0000002, dtype: int64

In [338]:
train0.ix[0]

102     0.0
103     0.0
201     0.0
301     1.0
503     0.0
506     0.0
601     1.0
702     0.0
704     0.0
705     0.0
706     0.0
707     0.0
708     0.0
801     1.0
1001    0.0
1002    0.0
1007    0.0
1009    0.0
1011    0.0
1012    0.0
1013    0.0
1017    0.0
1019    0.0
1020    0.0
1021    0.0
1022    0.0
1301    0.0
1302    0.0
1303    0.0
1304    0.0
       ... 
2601    0.0
2602    0.0
2701    0.0
2702    0.0
2703    0.0
2704    0.0
2705    0.0
2706    0.0
2707    0.0
3001    0.0
3101    0.0
3401    0.0
9991    0.0
9992    0.0
9993    0.0
1.1     0.0
1.2     0.0
1.3     0.0
1.4     0.0
1.5     1.0
2.1     0.0
2.2     0.0
2.3     0.0
2.4     0.0
2.5     1.0
3.1     1.0
3.2     0.0
3.3     0.0
3.4     0.0
3.5     0.0
Name: A0000002, dtype: float64

In [359]:
(target0_train.ix[0]- train0.ix[0]).sum()

1.0

Todo bien!! Podmeos entrenar

In [344]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
#Fix randome seed
seed=7
np.random.seed(seed)

In [345]:
print X0_train.shape
print y0_train.shape

(403709, 86)
(403709, 71)


In [348]:
input_dimen = X0_train.shape[1]
input_dimen

86

In [353]:
#Creare the model 
model=Sequential()
model.add(Dense(input_dimen, input_dim= 86, activation='relu') )
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(71, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_6 (Dense)                  (None, 86)            7482        dense_input_2[0][0]              
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 200)           17400       dense_6[0][0]                    
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 200)           40200       dense_7[0][0]                    
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 71)            14271       dense_8[0][0]                    
Total params: 79,353
Trainable params: 79,353
Non-trainable params: 0
_____________________

In [355]:
model.fit(X0_train, y0_train, validation_data =(X0_test, y0_test), nb_epoch=15, batch_size=128, verbose=1)
scores=model.evaluate(X0_test, y0_test, verbose=0)

print 'Accuracy: ', (scores[1]*100), '%'

Train on 403709 samples, validate on 100928 samples
Epoch 1/15
403709/403709 [==============================] - 20s - loss: 11.0902 - acc: 0.1716 - val_loss: 10.7952 - val_acc: 0.1473
Epoch 2/15
403709/403709 [==============================] - 19s - loss: 10.7152 - acc: 0.1941 - val_loss: 10.7484 - val_acc: 0.2351
Epoch 3/15
403709/403709 [==============================] - 19s - loss: 10.6786 - acc: 0.1945 - val_loss: 10.7333 - val_acc: 0.1660
Epoch 4/15
403709/403709 [==============================] - 19s - loss: 10.6572 - acc: 0.1832 - val_loss: 10.7020 - val_acc: 0.1964
Epoch 5/15
403709/403709 [==============================] - 19s - loss: 10.6421 - acc: 0.1656 - val_loss: 10.7063 - val_acc: 0.1713
Epoch 6/15
403709/403709 [==============================] - 20s - loss: 10.6310 - acc: 0.1553 - val_loss: 10.6994 - val_acc: 0.1392
Epoch 7/15
403709/403709 [==============================] - 20s - loss: 10.6215 - acc: 0.1506 - val_loss: 10.6913 - val_acc: 0.1478
Epoch 8/15
403709/403709